# Wjet Signal Region Shape Extraction Using Alpha Ratio Method

1. Calculate alpha using MC only
    1. Ratio of signal to side-band region
1. Get 1-sigma up/down alpha shape by varying the fit parameters by 1 sigma.
1. Grab corrected W+jet shape and normalization in side-band region from background estimation code.
1. Using alpha get the W+jet shape and normalization in signal region
1. Also, grab the corrected W+jet shape and its up/down from wjet fit parameters.

In [1]:
import ROOT as r
import math

%jsroot on

# To surpress unwanted messages
msgservice = r.RooMsgService.instance()
msgservice.setGlobalKillBelow(r.RooFit.FATAL)

import CMS_lumi, tdrstyle

tdrstyle.setTDRStyle()
r.gStyle.SetOptStat(0)

Welcome to JupyROOT 6.08/05


In [4]:
!ls /eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/WJets*.root

/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/WJetsToLNu_HT_100To200.root
/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/WJetsToLNu_HT_1200To2500.root
/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/WJetsToLNu_HT_200To400.root
/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/WJetsToLNu_HT_2500ToInf.root
/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/WJetsToLNu_HT_400To600.root
/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/WJetsToLNu_HT_600To800.root
/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/WJetsToLNu_HT_800To1200.root


In [64]:
#fin = r.TFile("Ntuples2/WWTree_VJets.root","read")
#fin = r.TFile("/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/Hadds_for_BkgEstimation/WWTree_VJets.root","read")
fin = r.TFile("/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/WJetsToLNu_HT_400To600.root","read")



treeIn = fin.Get("otree")

# Define the cuts
cutString_SideBand       =  "wSampleWeight*35867.06*btag0Wgt*genWeight*trig_eff_Weight*id_eff_Weight*pu_Weight*((type==0||type==1) && (l_pt2<0 && l_pt1>30 && (((type==0)&&(abs(l_eta1)<2.4)) || ((type==1)&&((abs(l_eta1)<2.5)&&!(abs(l_eta1)>1.4442 && abs(l_eta1)<1.566))))) && (((type==0)&&(pfMET_Corr>50)) || ((type==1)&&(pfMET_Corr>80))) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)&&(PuppiAK8_jet_tau2tau1<0.55)) && (nBTagJet_loose==0) && (vbf_maxpt_jj_m>800) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) && (((PuppiAK8_jet_mass_so_corr>40)&&(PuppiAK8_jet_mass_so_corr<65)) || ((PuppiAK8_jet_mass_so_corr>105)&&(PuppiAK8_jet_mass_so_corr<150))) && (mass_lvj_type0_PuppiAK8>600) && ((abs(ZeppenfeldWL_type0)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && ((abs(ZeppenfeldWH)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && (BosonCentrality_type0>1.0))"

cutString_Signal         =  "wSampleWeight*35867.06*btag0Wgt*genWeight*trig_eff_Weight*id_eff_Weight*pu_Weight*((type==0||type==1) && (l_pt2<0 && l_pt1>30 && (((type==0)&&(abs(l_eta1)<2.4)) || ((type==1)&&((abs(l_eta1)<2.5)&&!(abs(l_eta1)>1.4442 && abs(l_eta1)<1.566))))) && (((type==0)&&(pfMET_Corr>50)) || ((type==1)&&(pfMET_Corr>80))) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)&&(PuppiAK8_jet_tau2tau1<0.55)) && (nBTagJet_loose==0) && (vbf_maxpt_jj_m>800) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) && ((PuppiAK8_jet_mass_so_corr>65) &&(PuppiAK8_jet_mass_so_corr<105)) && (mass_lvj_type0_PuppiAK8>600) && ((abs(ZeppenfeldWL_type0)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && ((abs(ZeppenfeldWH)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && (BosonCentrality_type0>1.0))"

## Get W+jet histogram in signal, as well as side band region

In [65]:
hSignal_4bin = r.TH1D("hSignal_4bin","",4,600,2500)
hSignal_4bin.SetMarkerColor(1)
hSignal_4bin.SetLineColor(1)
hSignal_4bin.SetLineWidth(3)
hSignal_4bin.SetMarkerSize(1)

hSignal_15bin = r.TH1D("hSignal_15bin","",15,600,2500)
hSignal_15bin.SetMarkerColor(1)
hSignal_15bin.SetLineColor(1)
hSignal_15bin.SetLineWidth(3)
hSignal_15bin.SetMarkerSize(1)

hSignal_38bin = r.TH1D("hSignal_38bin","",38,600,2500)
hSignal_38bin.SetMarkerColor(1)
hSignal_38bin.SetLineColor(1)
hSignal_38bin.SetLineWidth(3)
hSignal_38bin.SetMarkerSize(1)

hSideBand_15bin = r.TH1D("hSideBand_15bin","",15,600,2500)
hSideBand_15bin.SetMarkerColor(2)
hSideBand_15bin.SetLineColor(2)
hSideBand_15bin.SetLineWidth(3)
hSideBand_15bin.SetMarkerSize(1)

hSideBand_38bin = r.TH1D("hSideBand_38bin","",38,600,2500)
hSideBand_38bin.SetMarkerColor(2)
hSideBand_38bin.SetLineColor(2)
hSideBand_38bin.SetLineWidth(3)
hSideBand_38bin.SetMarkerSize(1)

treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSignal_4bin",cutString_Signal)
treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSignal_15bin",cutString_Signal)
treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSignal_38bin",cutString_Signal)

treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSideBand_15bin",cutString_SideBand)
treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSideBand_38bin",cutString_SideBand)

317L

### Plot the Wjet histogram in signal region and side-band region

In [66]:
leg = r.TLegend(0.45,0.95,.95,0.75)

leg.AddEntry(hSignal_15bin,"signal region (MC)")
leg.AddEntry(hSideBand_15bin,"SideBand Region (MC)")

hSignal_15bin.SetStats(0)
hSideBand_15bin.SetStats(0)


c1 = r.TCanvas("c1", "canvas", 1000, 600)
c1.SetLogy()
c1.Divide(2,1)
c1.cd(1)
r.gPad.SetLogy()
hSignal_15bin.SetTitle("Log axis")
hSignal_15bin.Draw()
hSideBand_15bin.Draw("same")
leg.Draw()
c1.cd(2)
hSignal_15bin.SetTitle("Linear axis")
hSignal_15bin.Draw()
hSideBand_15bin.Draw("same")
leg.Draw()

c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


# Part-I

## Calculate alpha 

> by dividing W+jets MC in signal and side-band region


> fit the alpha

In [67]:
try: 
    Alpha.Delete();
except: 
    print "Alpha does not exists";
    
Alpha =  r.TH1D("Alpha","",15,600,2500)
Alpha = hSignal_15bin.Clone()
Alpha.Divide(hSideBand_15bin)
c1 = r.TCanvas("c1", "canvas", 800, 600)
Alpha.GetXaxis().SetTitle("mWW (GeV)")
Alpha.GetYaxis().SetTitle("alpha (Signal/Side-band)")
Alpha.SetMaximum(2.5)
Alpha.SetMinimum(-0.5)
Alpha.Draw()
f1 = r.TF1("f1","pol1",600,2500)
frp = Alpha.Fit("f1","S")
c1.Draw()
print "Integral = ",Alpha.Integral("width")
print "Integral = ",f1.Integral(600,2500)
#for i in range(1,39):
#    print i,",",Alpha.GetBinContent(i),",",Alpha.GetBinLowEdge(i),",",Alpha.GetBinWidth(i)

print "chi2 = ",f1.GetChisquare()
print "NDF = ",f1.GetNDF()
print "par0 = ",f1.GetParameter(0)
print "par1 = ",f1.GetParameter(1)

Alpha does not exists

****************************************
Minimizer is Linear
Chi2                      =      3.15019
NDf                       =            6
p0                        =      1.76731   +/-   0.328492    
p1                        = -0.000978455   +/-   0.000364771 
Integral =  994.434321871
Integral =  476.342461127
chi2 =  3.15018614393
NDF =  6
par0 =  1.76731188817
par1 =  -0.000978455051409


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [68]:
c1 = r.TCanvas("c1", "canvas", 1000, 700)

leg = r.TLegend(0.16,0.95,.65,0.75)
leg.SetNColumns(2);

hConf1s = Alpha.Clone("hConf1s");
hConf2s = Alpha.Clone("hConf2s");
hConf1s.Reset();
hConf2s.Reset();

r.TVirtualFitter.GetFitter().GetConfidenceIntervals(hConf1s, 0.68);
r.TVirtualFitter.GetFitter().GetConfidenceIntervals(hConf2s, 0.95);
hConf1s.SetStats(0);
hConf2s.SetStats(0);
hConf1s.SetMarkerSize(0);
hConf2s.SetMarkerSize(0);


hConf1s.SetFillColor(r.kGreen+1);
hConf2s.SetFillColor(r.kOrange);


hConf1s.SetLineColor(r.kGreen+1);
hConf2s.SetLineColor(r.kOrange);

hConf2s.SetTitle("")
hConf2s.SetMaximum(2.5)
hConf2s.Draw("e3");
hConf1s.Draw("e3 same");

Alpha.Draw("E1 same");
f1.Draw("same")
leg.AddEntry(Alpha,"MC","l")
leg.AddEntry(f1,"Fit","l")


leg.AddEntry(hConf1s,"1 #sigma band")
leg.AddEntry(hConf2s,"2 #sigma band")

leg.Draw()

print "chi2 = ",f1.GetChisquare()
print "NDF = ",f1.GetNDF()
print "par0 = ",f1.GetParameter(0)
print "par1 = ",f1.GetParameter(1)

pt = r.TPaveText(.65,.75,.98,.95,"brNDC");
pt.AddText("Chi2/ndf = "+str(round(f1.GetChisquare(),2))+"/"+str(f1.GetNDF()))
pt.AddText("a = "+str(round(f1.GetParameter(0),2))+" +/- "+str(round(f1.GetParError(0),2)))
pt.AddText("b = "+str(round(f1.GetParameter(1),4))+" +/- "+str(round(f1.GetParError(1),7)))

pt.Draw()

c1.Draw()

chi2 =  3.15018614393
NDF =  6
par0 =  1.76731188817
par1 =  -0.000978455051409


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
